In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark-3.0.3-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder\
                .appName("parquetFiles")\
                .config("spark.some.config.option","some-value")\
                .getOrCreate()

Apache Spark, veri işleme için büyük veri ortamında sıklıkla kullanılan bir hesaplama motorudur, ancak depolama sağlamaz, bu nedenle tipik bir senaryoda, veri işleme çıktısının harici bir depolama sisteminde depolanması gerekir. Spark SQL, dosya biçimi (CSV, JSON, metin, Parquet, ORC) veya JDBC gibi veri kaynakları için birkaç bağlayıcı sağlar. 2.4'ten beri Apache Avro için de destek vardır ve 3.0'dan beri ikili dosyalar için destek de okur. Ek olarak, bazılarını adlandırmak için MongoDB, ElasticSearch veya DynamoDB gibi diğer veri kaynaklarına bağlanmaya izin veren birkaç kitaplık vardır.
Parquet, Avro’dan farklı bir amaca sahiptir. Maksimum şema esnekliğine izin vermek yerine, sorgu hızlarını artırmak ve disk i/o’sunu azaltmak için kullanabileceğiniz şema türlerini optimize etmeye çalışır. Parquet, sütunlarda iç içe tiplere izin vererek geleneksel sütun bazlı tutmanın bazı zayıflıklarının üstesinden gelmeye çalışır. Böylece teknik olarak bir dizi olan bir sütuna ya da aslında birkaç sütun olan bir sütuna sahip olabilirsiniz.
Parquet, çok sayıda düşük seviye optimizasyona ve belgelerde bulabileceğiniz diskte nasıl depolandığı ile ilgili bir dizi ayrıntıya sahiptir. Parquet belki de Spark ile ilgili birçok projede göreceğiniz en yaygın dosya formatıdır

CSV gibi satır tabanlı dosyalara kıyasla, verilerin verimli bir şekilde sütun halinde depolanması için tasarlanmıştır.
sparksql parquet dosyalarını okumamızı ve yazmamızı sağlar
columnar formattadır orjinal datanın şemasını korur
Çok verimli sıkıştırma işlemi ve şemaları şifrelemeyi desteklemek için tasarlamıştır.
verileri sütün bazında depoladığı için alakasız verileri çok hızlı bir şekilde atabilir
columnar şekilde verileri depolayarak verilere daha hızlı şekilde ulaşabiliyorum ve donanım tasarrufu sağlayabilirim

In [3]:
dfjson=spark.read.load("people.json",format="json")#localimdeki json dosyasını okuyarak df oluşturdum
dfjson.write.save("people.parquet",format="parquet",mode ="ignore")
#df i şema bilgilerini koruyarak parquet dosyası halinde kaydedebilirim
parquetFile=spark.read.load("people.parquet",format="parquet")
#kaydettiğim people.parquet dosyasını okuyarak dfParquet adında df oluşturdum
#şimdi bu dfe sql atabilmek için tablo oluşturmam lazım
parquetFile.registerTempTable("parquetFile")
spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19").show()


+------+
|  name|
+------+
|Justin|
+------+



In [4]:
from pyspark.sql import Row
sc=spark.sparkContext

In [5]:
dfSquare=spark.createDataFrame(sc.parallelize(range(1,6)).map(lambda x :Row(single=x,double=x**2)))
dfSquare.show()
dfSquare.write.parquet("data/test_table/key=1",mode="ignore")
dfTrible=spark.createDataFrame(sc.parallelize(range(1,6)).map(lambda x :Row(single=x,triple=x**3)))
dfTrible.write.parquet("data/test_table/key=2",mode="ignore")
dfTrible.show()

+------+------+
|single|double|
+------+------+
|     1|     1|
|     2|     4|
|     3|     9|
|     4|    16|
|     5|    25|
+------+------+

+------+------+
|single|triple|
+------+------+
|     1|     1|
|     2|     8|
|     3|    27|
|     4|    64|
|     5|   125|
+------+------+



In [8]:
DF=spark.read.parquet("data/test_table")
DF.show()
DF.printSchema()

+------+------+---+
|single|double|key|
+------+------+---+
|     1|  null|  2|
|     2|  null|  2|
|     3|  null|  2|
|     4|  null|  2|
|     5|  null|  2|
|     1|     1|  1|
|     2|     4|  1|
|     3|     9|  1|
|     4|    16|  1|
|     5|    25|  1|
+------+------+---+

root
 |-- single: long (nullable = true)
 |-- double: long (nullable = true)
 |-- key: integer (nullable = true)



In [9]:
mergedDf=spark.read.option("mergeSchema", "true").parquet("data/test_table")
mergedDf.show()
mergedDf.printSchema()

+------+------+------+---+
|single|double|triple|key|
+------+------+------+---+
|     1|  null|     1|  2|
|     2|  null|     8|  2|
|     3|  null|    27|  2|
|     4|  null|    64|  2|
|     5|  null|   125|  2|
|     1|     1|  null|  1|
|     2|     4|  null|  1|
|     3|     9|  null|  1|
|     4|    16|  null|  1|
|     5|    25|  null|  1|
+------+------+------+---+

root
 |-- single: long (nullable = true)
 |-- double: long (nullable = true)
 |-- triple: long (nullable = true)
 |-- key: integer (nullable = true)



In [44]:
dfSquare.registerTempTable("dfSquare")
df3column=spark.sql("SELECT single,double,single*single*single as triple FROM dfSquare")
df3column.show()
df3column.printSchema()

+------+------+------+
|single|double|triple|
+------+------+------+
|     1|     1|     1|
|     2|     4|     8|
|     3|     9|    27|
|     4|    16|    64|
|     5|    25|   125|
+------+------+------+

root
 |-- single: long (nullable = true)
 |-- double: long (nullable = true)
 |-- triple: long (nullable = true)



spark sql daha iyi performans için parquet metadatasını önbelleğe alır 7
aynı şekilde eğer parquet dosyasındaki verileri tabloya çevirirsem tabloyuda önbelleğe alır
bu tablolar eğer güncelleniyorsa bunları yenilemek gerekir

In [ ]:
#spark.catalog.refreshTable("my_table") diyip güncelleyebilirim tabloyu